In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
fig_size = [16, 8]
plt.rcParams["figure.figsize"] = fig_size

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import random
import googlemaps
from datetime import datetime
import requests
import numpy as np
import networkx as nx
import math

def is_way_highway(way):
    return 'highway' in [x.attrib['k'] for x in way.findall("tag")]

def is_way_walkable(way):
    walkable_attribs = ['park', 'university', 'pitch', 'place_of_worship', 'glass', 'scrub']
    return any(flatten([[attrib in [tag.attrib['v'] for tag in way.findall("tag")]] 
                for attrib in walkable_attribs]))

# Haversine Distance in km between latitude and longitude pairs
def haversine_distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    return d

flatten = lambda l: [item for sublist in l for item in sublist]

In [14]:
# Data path
sample_path = "data/jiyugaoka.osm"


# Load osm data for xml preprocessing
import xml.etree.ElementTree

# Open original file
et = xml.etree.ElementTree.parse(sample_path)

e = et.getroot()

# Get list of all nodes and ways
node_list = e.findall("node")
way_list = e.findall("way")

# List of all tags
way_tag_list = flatten([[tag.attrib['k'] for tag in way.findall("tag")] for way in way_list])
node_tag_list = flatten([[tag.attrib['k'] for tag in node.findall("tag")] for node in node_list])

In [15]:
"""
# Remove missing way nodes (if exist)
n_ways = len(way_list)
node_id_list = [x.attrib['id'] for x in node_list]

for i, way in enumerate(way_list):
    print(str(i) + "/" + str(n_ways))
    for nd in list(way.iter('nd')):
        if not nd.attrib['ref'] in node_id_list:
            way.remove(nd)
            
et.write("data/sample_fixed.osm", xml_declaration=True)
"""


"""
# Filter out ways that are not walkable
for way in way_list:
    if not is_way_highway(way) and not is_way_park(way):
        e.remove(way)
        print("removed way " + str(way.attrib['id']))

et.write("data/sample_fixed.osm", xml_declaration=True)
"""



'\n# Remove missing way nodes (if exist)\nn_ways = len(way_list)\nnode_id_list = [x.attrib[\'id\'] for x in node_list]\n\nfor i, way in enumerate(way_list):\n    print(str(i) + "/" + str(n_ways))\n    for nd in list(way.iter(\'nd\')):\n        if not nd.attrib[\'ref\'] in node_id_list:\n            way.remove(nd)\n            \net.write("data/sample_fixed.osm", xml_declaration=True)\n'

In [16]:
# Convert to networkx graph

from lib.osm2nx import read_osm

# Read osm to networkx graph
G, rtree = read_osm(sample_path)

edge_iterator = iter(G.edges())
node_iterator = iter(G.nodes())

In [17]:
# DEFAULT COST FUNCTION FOR PATHFINDING
# Compute the shortest path between two sample nodes, using the cost function

def cost_function(node_1, node_2):
    origin_coords = (G.node[node_1]['lat'], G.node[node_1]['lon'])
    dest_coords = (G.node[node_2]['lat'], G.node[node_2]['lon'])
    dist =  haversine_distance(origin_coords, dest_coords)
    return dist


# AUGMENTED COST FUNCTION TO PREFER ALTERNATE PATHS
# Compute the shortest path between two sample nodes, using the modified cost function

def cost_function_green(node_1, node_2):
    
    distance_param = 0.1
    speed_param = 0.2
    lane_param = 0.3
    green_param = 0.4
    pol_param = 0.5
    
    origin_coords = (G.node[node_1]['lat'], G.node[node_1]['lon'])
    dest_coords = (G.node[node_2]['lat'], G.node[node_2]['lon'])
    dist =  haversine_distance(origin_coords, dest_coords)
    
    # If start and end node is same then just return distance
    if node_1==node_2:
        cost = dist
        
    # if nodes are isolated then just return distance
    elif len(G.edges(node_1))<1 or len(G.edges(node_2))<1:
        cost = dist
        
    else:

        # collect edges for both nodes
        node_1_edges = [G.get_edge_data(edge[0], edge[1]) for edge in G.edges(node_1)]
        node_2_edges = [G.get_edge_data(edge[0], edge[1]) for edge in G.edges(node_2)]

        # compute aggregates values of node based on adjacent edge parameters
        # max of all adjacent edges is assigned to node value
        node_1_lane_agg = np.max([float(x['lane']) for x in node_1_edges])
        node_1_green_agg = np.max([float(x['green']) for x in node_1_edges])
        node_1_speed_agg = np.max([float(x['speed']) for x in node_1_edges])
        node_1_pol_agg = np.max([float(x['pol_max']) for x in node_1_edges])

        # compute aggregates values of node based on adjacent edge parameters
        # max of all adjacent edges is assigned to node value
        node_2_lane_agg = np.max([float(x['lane']) for x in node_2_edges])
        node_2_green_agg = np.max([float(x['green']) for x in node_2_edges])
        node_2_speed_agg = np.max([float(x['speed']) for x in node_2_edges])
        node_2_pol_agg = np.max([float(x['pol_max']) for x in node_2_edges])
        
        # Summation of start and end node attributes as taken as approximation of path values
        lane_value = node_1_lane_agg + node_2_lane_agg
        green_value = node_1_green_agg + node_2_green_agg
        speed_value = node_1_speed_agg + node_2_speed_agg
        pol_value = node_1_pol_agg + node_2_pol_agg
        
        # Final cost computed by regression via fixed parameters
        cost = dist*distance_param + speed_value*speed_param + lane_value*lane_param + green_value*green_param + pol_value*pol_param
    
    return cost

In [18]:
# Test with two random nodes

node_1 = '-258'
node_2 = '-3835'

normal_path = nx.astar_path(G, node_1, node_2, heuristic=cost_function)
green_path = nx.astar_path(G, node_1, node_2, heuristic=cost_function_green)

In [19]:
# Data preparation for plotting

path_coords = [G.node[n] for n in normal_path]
path_coords_xy = [(x['lat'], x['lon']) for x in path_coords]
lats, lons = zip(*path_coords_xy)

# Data preparation for plotting (for green path)

green_path_coords = [G.node[n] for n in green_path]
green_path_coords_xy = [(x['lat'], x['lon']) for x in green_path_coords]
g_lats, g_lons = zip(*green_path_coords_xy)

In [25]:
# Plot on google maps

# google maps api key here
# gmap_key = <API KEY>

from gmplot import gmplot

# Place map
gmap = gmplot.GoogleMapPlotter(lats[0], lons[0], 13, apikey=gmap_key)

gmap.plot(lats, lons, 'cornflowerblue', edge_width=10)

gmap.plot(g_lats, g_lons, 'green', edge_width=10)

# Draw
gmap.draw("out/map.html")

In [26]:
cost_function(node_1, node_2)

0.02365022434441046

In [27]:
cost_function_green(node_1, node_2)

50.45466502243445

In [ ]:
# Demo

import numpy as np
from sklearn.metrics.pairwise import cosine_distances

node_id_list = list(G.nodes())
lat_lons = [G.node[i] for i in node_id_list]
nn_vector = [(x['lat'], x['lon']) for x in lat_lons]

# compute nearest node and reutrn nearest node id and latitude and longitude

def get_nearest_point(lat, lon):
    query = (lat, lon)
    distances = [haversine_distance(query, dest_coords) for dest_coords in nn_vector]
    nearest_id = np.argsort(distances)[0]
    nn_id = node_id_list[nearest_id]
    return nn_id

start_lat = 35.593800
start_lon = 139.695050
   
end_lat = 35.614615
end_lon = 139.670745

# Test with two random nodes

start_id = get_nearest_point(float(start_lat), float(start_lon))
end_id = get_nearest_point(float(end_lat), float(end_lon))

node_1 = start_id
node_2 = end_id

normal_path = nx.astar_path(G, node_1, node_2, heuristic=cost_function)
green_path = nx.astar_path(G, node_1, node_2, heuristic=cost_function_green)

# Data preparation for plotting

path_coords = [G.node[n] for n in normal_path]
path_coords_xy = [(x['lat'], x['lon']) for x in path_coords]
lats, lons = zip(*path_coords_xy)

# Data preparation for plotting (for green path)

green_path_coords = [G.node[n] for n in green_path]
green_path_coords_xy = [(x['lat'], x['lon']) for x in green_path_coords]
g_lats, g_lons = zip(*green_path_coords_xy)

# Plot on google maps

# google maps api key here
gmap_key = ''

from gmplot import gmplot

# Place map
gmap = gmplot.GoogleMapPlotter(lats[0], lons[0], 13, apikey=gmap_key)

gmap.plot(lats, lons, 'cornflowerblue', edge_width=10)

gmap.plot(g_lats, g_lons, 'green', edge_width=10)

# Draw
#gmap.draw("map.html")

out_path = "out/output.html"
gmap.draw(out_path)
print("Map saved to: " + out_path)